### 1. Import Data

File “StrainTemperature.csv” contains a dataset, with 7 columns and 337 rows.  
- Col. 1 lists the strain measured in a structural member, in micro-strains, με (i.e., in parts per million).  
- Col. 2 to 7 list the temperature measured by 6 thermometers in different locations, in degree Celsius.  
Each row refers to a specific time when all measures (of strain and temperature) are collected.
Measures are collected every 30 minutes for one week (hence the rows are 337 = 7 × 24 × 2 + 1).

In [11]:
import pandas as pd

# 데이터 로드
data = pd.read_csv("./data/StrainTemperature.csv")

# 데이터 구조 확인
print(data.head())  # 데이터의 첫 5행
print(data.info())  # 데이터 유형 및 결측치 확인
print(data.describe())  # 기본 통계 값


    69.754  22.681  23.836  24.512  25.141   23.65  24.048
0   98.703  23.317  24.357  25.073  25.689  24.205  24.622
1  104.404  23.945  24.926  25.564  26.028  24.741  25.121
2  101.514  24.226  25.503  25.994  26.164  25.146  25.481
3   99.808  24.432  26.114  26.177  26.272  25.443  25.765
4  105.260  24.612  26.233  26.532  26.378  25.656  25.955
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   69.754  336 non-null    float64
 1   22.681  336 non-null    float64
 2   23.836  336 non-null    float64
 3   24.512  336 non-null    float64
 4   25.141  336 non-null    float64
 5   23.65   336 non-null    float64
 6   24.048  336 non-null    float64
dtypes: float64(7)
memory usage: 18.5 KB
None
           69.754      22.681      23.836      24.512      25.141       23.65  \
count  336.000000  336.000000  336.000000  336.000000  336.000000  336.000000   
m

### 2. Data Proprecessing

결측값이 있는지 확인

In [12]:
# 결측값 확인
print(data.isnull().sum())

# 결측값 대체 또는 제거
data = data.dropna()  # 결측값 제거 (필요시 다른 방법으로 대체)


69.754    0
22.681    0
23.836    0
24.512    0
25.141    0
23.65     0
24.048    0
dtype: int64


### 3. Calibrate a linear regression model

Using all 6 temperatures and also a “constant feature”, to infer the strain as a function of the temperatures.   
What is the coefficient of determination 𝑅^2 for this fitted model?  
Briefly explain how the value of 𝑅^2 is related to the uncertainty in inferring the strain.

6개의 온도와 함께 "상수 항(constant feature)"을 사용하여, 변형률을 온도의 함수로 추정하십시오.
이 모델에 대한 결정계수 R^2 값은 무엇입니까?
𝑅^2 값이 변형률 추정의 불확실성과 어떻게 관련이 있는지 간단히 설명하십시오.

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# 입력(X)와 출력(y) 분리
X = data.iloc[:, 1:]  # 온도 데이터 (2~7열)
y = data.iloc[:, 0]   # 변형률 (1열)

# 데이터 분리 (학습/테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print("R^2 Score:", r2_score(y_test, y_pred))


R^2 Score: 0.9331336033622016


1. 결정계수 R^2값  
모델의 결정계수 R^2 값은 0.9331입니다.
이는 테스트 데이터의 변동성 중 약 93.31%가 이 선형 회귀 모델로 설명된다는 것을 의미합니다.
2. R^2값과 불확실성의 관계  
R^2 값은 모델이 입력 데이터(온도)로부터 출력 데이터(변형률)를 얼마나 잘 설명할 수 있는지를 나타냅니다.
높은 R^2값(1에 가까운 값)은 모델이 데이터의 변동성을 잘 설명하며, 예측 결과의 불확실성이 작다는 것을 의미합니다.
낮은 R^2값은 모델이 데이터를 충분히 설명하지 못해, 예측 결과의 불확실성이 더 크다는 것을 나타냅니다.
결론  
R^2 값이 0.9331로 높기 때문에, 이 모델은 변형률을 온도의 함수로 효과적으로 추정하고 있으며, 불확실성이 상대적으로 낮습니다. 하지만, 
R^2 값만으로 모델의 품질을 완전히 평가할 수는 없으므로, 잔차 분석 및 다중공선성 문제도 추가로 고려해야 합니다.

### 4. Build a 95% confidence interval for each of the parameters

Briefly summarize the meaning of such intervals and discuss why are “large” (if they are indeed large).

온도와 변형률 간의 관계를 나타내는 각 매개변수에 대해 95% 신뢰구간을 구축하십시오.  
그러한 신뢰구간의 의미를 간단히 요약하고, 만약 신뢰구간이 "크다면" 그 이유를 논의하십시오.

In [14]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

# 데이터 준비 (독립 변수와 종속 변수 설정)
X = data.iloc[:, 1:]  # 온도 데이터 (2~7열)
y = data.iloc[:, 0]   # 변형률 (1열)

# 상수 항 추가 (Statsmodels는 상수 항을 수동으로 추가해야 함)
X = sm.add_constant(X)

# 선형 회귀 모델 피팅
model = sm.OLS(y, X).fit()

# 결과 출력
print(model.summary())

# 95% 신뢰구간 추출
confidence_intervals = model.conf_int(alpha=0.05)  # 95% 신뢰구간
confidence_intervals.columns = ['Lower Bound', 'Upper Bound']
confidence_intervals.index = ['Constant'] + list(data.columns[1:])  # 변수 이름 추가

print("\n95% Confidence Intervals:")
print(confidence_intervals)


                            OLS Regression Results                            
Dep. Variable:                 69.754   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     751.1
Date:                Tue, 07 Jan 2025   Prob (F-statistic):          1.17e-188
Time:                        02:24:59   Log-Likelihood:                -1480.2
No. Observations:                 336   AIC:                             2974.
Df Residuals:                     329   BIC:                             3001.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -319.7674     14.606    -21.894      0.0

1. 95% 신뢰구간  
결과에서 각 변수의 95% 신뢰구간은 마지막 두 열 [0.025, 0.975]에 표시됩니다.  

예시:  
const (상수 항)의 신뢰구간: [-348.499, -291.035]  
첫 번째 변수(22.681)의 신뢰구간: [-13.814, 30.389]  
마지막 변수(24.048)의 신뢰구간: [37.327, 184.359]  
신뢰구간의 의미는 해당 계수가 95% 확률로 이 범위 내에 존재할 것이라는 것을 나타냅니다.  

2. 계수의 유의성  
P>|t| 값은 각 변수의 계수가 통계적으로 유의미한지 여부를 나타냅니다.  
일반적으로   
p-값이 0.05보다 작으면 해당 계수가 유의미하다고 판단합니다.  
결과 분석:  
const (상수 항): p=0.000, 유의미.  
마지막 변수(24.048): p=0.003, 유의미.  
다른 변수들은 p>0.05로 유의미하지 않음.  

3. 신뢰구간 크기  
신뢰구간이 크다는 것은 해당 계수 추정치의 불확실성이 크다는 것을 의미합니다.  
예를 들어:  
첫 번째 변수(22.681)의 신뢰구간은 **[-13.814, 30.389]**로 넓은 범위를 가집니다.  
이는 모델이 해당 변수의 영향을 정확히 추정하지 못하고 있음을 나타냅니다.  

4. 왜 신뢰구간이 클까?  
다중공선성: 독립 변수들 간에 상관관계가 높으면, 특정 변수의 계수를 안정적으로 추정하기 어려워집니다.  
데이터 품질: 데이터의 샘플 크기가 작거나, 노이즈가 많을 경우 추정값의 불확실성이 커질 수 있습니다.  
변수의 낮은 설명력: 해당 변수가 종속 변수(변형률)에 미치는 영향이 크지 않을 경우.  

### 5. Investigate the issue of “multicollinearity.” Compute the Variance Inflation Factor (VIF) for each of the temperatures

To do so, you need to regress the temperatures recorded by each thermometer 𝑖 against those by all
other thermometers, compute the coefficient of determination of that regression, 𝑅^2, and then use formula []. Briefly summarize the meaning of VIFs and draw some conclusions
about the temperatures being “independent” features or, on the contrary, “redundant” features.

이제 다중공선성(multicollinearity) 문제를 조사하십시오. 각 온도 변수에 대해 **분산 팽창 요인(VIF, Variance Inflation Factor)**을 계산하십시오.  
VIF에 대한 설명은 다음 링크에서 확인할 수 있습니다: https://en.wikipedia.org/wiki/Variance_inflation_factor.   
이를 수행하려면, 각 온도계 i에 대해 해당 온도를 나머지 온도들에 대해 회귀(regress)하고, 해당 회귀의 결정계수 R^2 를 계산하십시오.   
그런 다음, 다음 공식을 사용하여 VIF를 계산하십시오:  
VIF의 의미를 간단히 요약하고, 온도가 "독립적인 특성(independent features)"인지, 또는 "중복된 특성(redundant features)"인지에 대한 결론을 도출하십시오.

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 데이터 준비 (온도 변수만 선택)
X = data.iloc[:, 1:]  # 온도 데이터 (2~7열)
X_columns = X.columns

# VIF 계산
vif_values = []

for i in range(X.shape[1]):
    # 현재 변수(i)를 종속 변수로 설정, 나머지를 독립 변수로 설정
    y = X.iloc[:, i]
    X_temp = X.drop(X.columns[i], axis=1)
    
    # 선형 회귀 모델 학습
    model = LinearRegression()
    model.fit(X_temp, y)
    
    # 결정계수 R^2 계산
    r_squared = model.score(X_temp, y)
    
    # VIF 계산
    vif = 1 / (1 - r_squared)
    vif_values.append(vif)

# VIF 값을 데이터프레임으로 정리
vif_data = pd.DataFrame({
    "Feature": X_columns,
    "VIF": vif_values
})

print(vif_data)


  Feature           VIF
0  22.681   1678.481395
1  23.836   2416.019796
2  24.512   3142.669321
3  25.141   1831.153298
4   23.65   4971.569075
5  24.048  25200.370504


In [16]:
# 라이브러리 사용 (검증증)
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# 데이터 준비 (온도 변수만 선택)
X = data.iloc[:, 1:]  # 온도 데이터 (2~7열)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns  # 변수 이름
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# 결과 출력
print("Variance Inflation Factor (VIF):")
print(vif_data)


Variance Inflation Factor (VIF):
  Variable            VIF
0   22.681   43134.207218
1   23.836   47878.049733
2   24.512   45798.432010
3   25.141   29565.418107
4    23.65   85779.767266
5   24.048  481470.634236


VIF(Variance Inflation Factor)는 특정 독립 변수가 다른 독립 변수들과 얼마나 상관되어 있는지를 측정하는 지표입니다. VIF 값이 높을수록 해당 변수는 다른 변수들과 강한 상관관계를 가지며, 독립적이라기보다는 중복된 특성으로 작용하고 있음을 나타냅니다. 일반적으로 VIF 값이 10을 초과하면 다중공선성(multicollinearity) 문제가 있다고 간주됩니다.

현재 분석 결과에서 모든 온도 변수의 VIF 값이 매우 높게 나타났습니다(최소 29,565에서 최대 481,470). 이는 모든 온도 변수들이 서로 높은 상관관계를 가지고 있으며, "독립적인 특성(independent features)"이 아니라 "중복된 특성(redundant features)"으로 간주될 수 있음을 의미합니다. 이러한 중복성은 회귀 모델에서 변수들의 개별적인 영향을 정확히 추정하는 것을 어렵게 만들며, 모델의 안정성과 신뢰성을 저하시킬 가능성이 있습니다.

### 6. address and “solve” the issue of multicollinearity, by modifying the model and/or the set of inputs.

#### 6.1 다중공선성을 해결해야 하는 이유
다중공선성을 해결하는 이유는 여러 가지가 있을 수 있으며, 현재 분석에서는 아래와 같은 이유가 모두 관련될 수 있습니다:

1) 변형률 추정의 개선
다중공선성은 독립 변수 간 높은 상관관계로 인해 모델이 각 변수의 개별적인 영향을 정확히 추정하지 못하게 만듭니다. 이를 해결하면 변형률 추정의 정확도를 높이고 모델의 예측 성능을 개선할 수 있습니다.

2) 더 단순한 모델 획득
다중공선성이 존재하면 불필요하게 중복된 변수를 포함하게 되어 모델이 복잡해집니다. 이러한 변수를 제거하거나 차원 축소 기법을 적용하면 모델이 단순화되면서도 성능을 유지하거나 향상시킬 수 있습니다.

3) 변형률과 특정 온도 간 관계 이해
다중공선성이 있으면 변수들 간 상관관계로 인해 특정 변수의 개별적인 영향을 해석하기 어려워집니다. 다중공선성을 줄이면 변형률에 특정 온도가 얼마나 영향을 미치는지 더 명확히 이해할 수 있습니다.

4) 모델 매개변수의 불확실성 감소
다중공선성은 회귀 모델의 매개변수를 불안정하게 만들어, 매개변수 추정치의 신뢰구간을 넓게 하고 모델의 예측 신뢰도를 저하시킵니다. 이를 해결하면 매개변수 추정의 불확실성을 줄이고 모델의 안정성을 높일 수 있습니다.

결론적으로, 다중공선성 문제를 해결하는 것은 모델의 성능을 개선하고, 해석 가능성을 높이며, 모델 매개변수의 안정성과 신뢰성을 확보하기 위한 중요한 과정입니다.

#### 6.2 다중공선성 해결을 위한 기법 1) Subset

VIF가 높은 변수 제거

In [17]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF 계산 함수
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

# 초기 VIF 계산
vif_before = calculate_vif(X)
print("VIF Before:")
print(vif_before)

# VIF가 가장 높은 변수 제거 (예: "24.048")
X_subset = X.drop("24.048", axis=1)

# 수정된 데이터로 모델 재학습
X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# R^2 결과 출력
print("R^2 Score After Subset Selection:", r2_score(y_test, y_pred))

# 새로운 VIF 계산
vif_after = calculate_vif(X_subset)
print("VIF After:")
print(vif_after)


VIF Before:
  Feature            VIF
0  22.681   43134.207218
1  23.836   47878.049733
2  24.512   45798.432010
3  25.141   29565.418107
4   23.65   85779.767266
5  24.048  481470.634236
R^2 Score After Subset Selection: 0.9999582081308571
VIF After:
  Feature           VIF
0  22.681   8361.708108
1  23.836  13648.207919
2  24.512   9068.252530
3  25.141    345.288694
4   23.65  84211.420734


#### 6.3 다중공선성 해결을 위한 기법 2) PCA  


PCA를 사용해 차원을 축소하고, 다중공선성을 제거한 새로운 변수로 모델을 학습

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA 적용 (주성분 수 설정, 예: 2개)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# PCA 이후 모델 학습
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# R^2 결과 출력
print("R^2 Score After PCA:", r2_score(y_test, y_pred))

# PCA 변동성 설명 비율
print("Explained Variance Ratio:", pca.explained_variance_ratio_)


R^2 Score After PCA: 0.99995625124426
Explained Variance Ratio: [0.98406508 0.0133762 ]


#### 6.4 다중공선성 해결을 위한 기법 3) Ridge Regression

In [19]:
from sklearn.linear_model import Ridge

# 릿지 회귀 모델
ridge = Ridge(alpha=1.0)  # 알파 값은 정규화 강도를 조절
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

# R^2 결과 출력
print("R^2 Score with Ridge Regression:", r2_score(y_test, y_pred_ridge))


R^2 Score with Ridge Regression: 0.999954993143469


#### 6.5 다중공선성 해결을 위한 기법 4) Lasso Regression

In [20]:
from sklearn.linear_model import Lasso

# 라쏘 회귀 모델
lasso = Lasso(alpha=0.1)  # 알파 값은 정규화 강도를 조절
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

# R^2 결과 출력
print("R^2 Score with Lasso Regression:", r2_score(y_test, y_pred_lasso))


R^2 Score with Lasso Regression: 0.9996497233047609



**6.2 Subset Selection**
방법:
- VIF가 가장 높은 변수(24.048)를 제거하고 모델을 재학습.
결과:
- R²: **0.9999**로 여전히 매우 높은 설명력을 유지.
- VIF 값: 다른 변수들의 VIF 값이 여전히 높으나, 일부 감소를 확인.

결론:
- 특정 변수를 제거하는 방식으로 다중공선성을 일부 완화할 수 있지만, 완벽히 해결되지는 않음. 추가적인 변수 제거 또는 다른 기법과의 조합이 필요.

---

**6.3 PCA (Principal Component Analysis)**
방법:
- 주성분 분석(PCA)을 통해 온도 변수의 차원을 축소(2개의 주요 성분으로 변환).
결과:
- R²: **0.9999**로 높은 설명력을 유지.
- Explained Variance Ratio: 첫 번째 두 개 주성분이 **99% 이상**의 변동성을 설명.

결론:
- PCA는 다중공선성을 효과적으로 제거하며, 변수 수를 줄이고도 모델의 설명력을 유지.
- 다중공선성을 해결하면서도 모델 복잡성을 줄이는 데 유리.

---

**6.4 Ridge Regression**
방법:
- 릿지 회귀를 통해 정규화를 적용.
- 알파 값: 1.0.
결과:
- R²: **0.9999**로 높은 설명력을 유지.

결론:
- 릿지 회귀는 다중공선성 문제를 해결하는 동시에 모델 성능을 유지.
- 변수 간의 영향을 모두 유지하면서도 가중치를 안정화.

---

### **6.5 Lasso Regression**
방법:
- 라쏘 회귀를 통해 정규화와 변수 선택을 동시에 수행.
- 알파 값: 0.1.
결과:
- R²: **0.9999**로 높은 설명력을 유지.

결론:
- 라쏘 회귀는 불필요한 변수를 자동으로 제거하며, 모델 단순화와 다중공선성 문제를 동시에 해결.
- 알파 값에 따라 선택되는 변수가 달라질 수 있음.

---

**전체 결론**
1. **Subset Selection**:
   - 변수 제거로 다중공선성을 완화할 수 있으나, 가장 간단한 접근법.
   - 변수 제거는 데이터 해석에 영향을 줄 수 있음.

2. **PCA**:
   - 다중공선성을 완전히 제거하며, 모델의 설명력을 유지.
   - 차원을 축소하므로 모델 해석력이 떨어질 수 있음.

3. **Ridge Regression**:
   - 다중공선성을 해결하면서 모든 변수를 유지.
   - 해석 가능성이 높음.

4. **Lasso Regression**:
   - 다중공선성 해결과 변수 선택을 동시에 수행.
   - 가장 해석 가능하고 간단한 모델을 생성할 수 있음.


장단점 비교 및 결론

### 7. Predict Future

 선형 회귀를 사용하여, 현재 (또는 과거)의 온도와 변형률을 함수로 하여 미래의 변형률을 예측하는 모델을 개발하십시오. (미래 시점에서 수집된 데이터를 사용할 수 없습니다.)

이전 질문에서 제안한 모델과 관련하여, 예측의 정확도를 정량화하십시오.
미래 변형률에 대해 95% 신뢰구간을 어떻게 정의할 수 있습니까?
이를 답하기 위해, 연속된 시점에서 변형률에 영향을 미치는 노이즈가 상관되어 있을 수 있음을 고려하십시오.

노이즈 상관성 문제를 분석하고 이것이 초래하는 결과를 설명하며, 미래 변형률에 대한 신뢰구간을 정의할 때 이 현상을 어떻게 반영할 수 있을지 논의하십시오.

다중공선성이 마지막 두 질문에서 정의된 미래 변형률 예측과 어떻게 관련되어 있는지 논의